### Solution Thermal Electric Problem
- load term physical enviroment
- Stepwise solving

In [ ]:
# Importiere benötigte Bibliotheken
import os
import pyansys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib as mpl
from matplotlib.ticker import MultipleLocator
from IPython.display import Image
from scipy.interpolate import interp1d
import time as zeit

In [ ]:
# cwd change
def Newdir(path):
    import os
    # new dir
    try:
        os.makedirs(path)
    except FileExistsError:
        print (path + '\t directory already exists!')
    # cwd
    os.chdir(path)
    print('cwd:', os.getcwd())

subfolder = 'Output'

In [ ]:
def SaveAndExit():
    ansys.save()
    ansys.Exit()

In [ ]:
# ansys apdl aktivieren
filecode = 'therm_cdw_ring_al_py'           # TODO
cwd  =  'F:/SHK/Solution/'                    # TODO
mkpath = cwd + filecode
Newdir(mkpath)
# physiscs name
PHYS_STRUCT       = 'Struct'
PHYS_T_E          = 'T_E'
STRUCT_FILE       = 'Structure'
T_E_FILE          = 'Thermo_Electric'
PRMTR_DT = 'PRMTR_DT'
LRNTZ_NMBR        = 2.44E-8
CNTCT1_LYR_THCKNSS= 0.00005
CNTCT1_SRFC_CNDTN = 1
ansys = pyansys.Mapdl(run_location = os.getcwd(),jobname = T_E_FILE, interactive_plotting = True)
ansys.resume('{}/{}'.format(mkpath,STRUCT_FILE),'db')

In [ ]:
# all of the data show
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
df5 = pd.read_csv('{}/{}/{}.csv'.format(mkpath,subfolder,'Press_Time'),index_col=0)
df5

#### 7.1 load term physical enviroment

In [ ]:
ansys.assign('esav',PHYS_T_E,'esav')
ansys.assign('emat',PHYS_T_E,'emat')
with ansys.non_interactive:
    ansys.run('save')
ansys.input('{}/{}'.format(mkpath,PHYS_T_E),'ph2' )
ansys.parres('new',PRMTR_DT,'sav')
ansys.allsel('all')
ansys.load_parameters()
ansys.parameters

In [ ]:
ansys.input('{}/{}'.format(mkpath,PRMTR_DT),'sav')
zeit.sleep(1)
ansys.load_parameters()
ansys.parameters

In [ ]:
def table_save(dfname,list1,list2,tread=False,t_name = None):
    df =pd.DataFrame({0:list1,1:list2},index=None) 
    df.sort_values(0,axis = 0,ascending = True,inplace = True)
    df.to_csv('{}/{}/{}.csv'.format(mkpath,subfolder,dfname),index=None,header=None)
    if tread == True:
        ansys.finish()
        ansys.tread('CNTCT1_{}_X'.format(t_name),'{}/{}/{}'.format(mkpath,subfolder,dfname),'csv')
        
def list_clear():
    ECR_l.clear()
    TCC_l.clear()
    ECC_l.clear()
#     NX_l.clear()

ECR_l = []
ECC_l = []
TCC_l = []
NX_l = []
DT_I  = .1E-3
# NMBR_SS_PF =ansys.parameters['NMBR_SS_PF']
# NMBR_SS_SP = ansys.parameters['NMBR_SS_SP']
NMBR_SS_SWF = ansys.parameters['NMBR_SS_SWF'] 
SS_CUR=ansys.parameters['NMBR_SS_SWF']                        # Start at ignition
LS_CUR=3
NNUM = list(df5.index)
time_l = list(df5)

#### Stepwise solving

In [ ]:
for i in range(len(time_l)):
    if i ==0:
        ansys.prep7()
        ansys.allsel('all')
        # Update displaced structure for time at start of current flow
        ansys.upgeom(1,LS_CUR,SS_CUR,STRUCT_FILE,'rst')
        ansys.slashsolu()
        ansys.r(5,'',1.0,0.1,0,)
        ansys.rmore('','',1.0E20,0.0,1.0,)
        ansys.rmore(0.0,99999999999,1.0,'',1.0,0.5)
        ansys.rmore(9999999999999,1.0,1.0,0.0,'',1.0)
        ansys.rmore('','','','',)
        ansys.rmore('','','',913)
        ansys.r(7,'',1.0,0.1,0,)
        ansys.rmore('','',1.0E20,0.0,1.0,)
        ansys.rmore(0.0,99999999999,1.0,'',1.0,0.5)
        ansys.rmore(9999999999999,1.0,1.0,0.0,'',1.0)
        ansys.rmore('','','','',)
        ansys.rmore('','','',913)
    else:
        ansys.slashsolu()
        ansys.antype(4,'restart',i,'CONTINUE')
        ansys.input('{}/{}'.format(mkpath,PRMTR_DT),'sav')
        zeit.sleep(1)
        ansys.prep7()
        ansys.allsel('all')
        ansys.upgeom(-1,LS_PREV,SS_PREV,STRUCT_FILE,'rst')
        ansys.upgeom(1,LS_CUR,SS_CUR,STRUCT_FILE,'rst')
        ansys.post1()
        ansys.lsel('s','','',200,202,1)
        ansys.nsll('s',1)
        ansys.esln('s',0)
        ansys.nsle('r','corner')
        time = time_l[i]
        for j in range(len(NNUM)):
            if i == 1:
                ansys.get('NX', 'node', NNUM[j], 'loc', 'x')
                ansys.load_parameters()
                NX = ansys.parameters['NX']
                NX_l.append(NX)
                
            ansys.get('CNTCT1_ND_TEMP','node',NNUM[j],'temp')
            ansys.load_parameters()
            NTEMP = ansys.parameters['CNTCT1_ND_TEMP'] 
            # compare yeild between two Mat at 298K
            ansys.run('YIELD_MAT1 = TABLE_YIELD_MAT1({})'.format(NTEMP))
            ansys.run('YIELD_MAT2 = TABLE_YIELD_MAT2({})'.format(NTEMP))
            ansys.load_parameters()
            yield_min = min(ansys.parameters['YIELD_MAT1'],ansys.parameters['YIELD_MAT2'])
            # sum RSVX of two Mat at 298K
            ansys.run('RSVX_MAT1 = TABLE_RSVX_MAT1({})'.format(NTEMP))
            ansys.run('RSVX_MAT2 = TABLE_RSVX_MAT2({})'.format(NTEMP))
            ansys.load_parameters()
            RSVX = sum([ansys.parameters['RSVX_MAT1'],ansys.parameters['RSVX_MAT2']])/2       
            press = df5.loc[NNUM[j],time]
            if press ==0:
                ECR = np.nan  
                ECC =0  
            else:
                ECR = 3 * 50 * CNTCT1_LYR_THCKNSS * ((yield_min / press)**CNTCT1_SRFC_CNDTN) * (RSVX)
                ECC = 1/ECR    
            TCC = LRNTZ_NMBR*ECC*NTEMP
            ECR_l.append(ECR)
            ECC_l.append(ECC)
            TCC_l.append(TCC)
        # save as csv and write into Ansys table
        table_save('ECR_X_{}'.format(time),NX_l,ECR_l)
        table_save('ECC_X_{}'.format(time),NX_l,ECC_l,True,'ECC')
        table_save('TCC_X_{}'.format(time),NX_l,TCC_l,True,'TCC')
        # clear lists
        list_clear()
        ansys.allsel('all')
    # ----------------------------------------------------------------------------------------------------------------------------------
    # ----------------------------------------------------------------------------------------------------------------------------------
    # write condutcivities to real constants of contact elements
    ansys.slashsolu()
    ansys.r(3,'',1.0,0.1,0,)
    ansys.rmore('','',1.0E20,0.0,1.0,)
    ansys.rmore(0.0,'%CNTCT1_TCC_X%',1.0,'',1.0,0.5)
    ansys.rmore('%CNTCT1_ECC_X%',1.0,1.0,0.0,'',1.0)
    ansys.rmore('','','','',)
    ansys.rmore('','','',913)

    if i > 0:
        ansys.fk(410,'amps','%TABLE_I({})%'.format(time))
    else:
        ansys.fk(410,'amps',100)
    ansys.time((i+1)*DT_I)

    if i ==0:
        SS_PREV=NMBR_SS_SWF
        LS_PREV=3
        SS_CUR=1
        LS_CUR=4
    if i ==1:
        SS_PREV=1
        LS_PREV=4
        SS_CUR =2
    if i > 1:
        SS_PREV+=1
        SS_CUR+=1

    ansys.allsel('all')
    ansys.autots('OFF')
    ansys.save()
    ansys.solve()
    ansys.finish()
    ansys.assign('esav')
    ansys.assign('emat')
    ansys.post1()
    ansys.plnsol('temp','',0)
    ansys.finish()

    
ansys.prep7()
ansys.upgeom(-1,LS_PREV,SS_PREV,STRUCT_FILE,'rst')

In [ ]:
SaveAndExit()

### Plot Result

In [ ]:
def get_time(files, col_list):
    for i in range(len(files)):
        a = files[i].split('_')[2].split('.')[:2]
        time = '{}.{}'.format(a[0],a[1])
        col_list.append(time)

files = [file for file in os.listdir('{}/{}/'.format(mkpath,subfolder)) if file.split('_')[0] == 'ECR']
all_ECR_data = pd.DataFrame()
columns = []
get_time(files,columns)

for file in files:
    df = pd.read_csv('{}/{}/{}'.format(mkpath,subfolder,file), header=None)
    all_ECR_data = pd.concat([all_ECR_data,df[1]],axis=1,)

all_ECR_data.columns = columns 
all_ECR_data ['x'] = df[0]
all_ECR_data = all_ECR_data.set_index('x')
all_ECR_data = all_ECR_data.replace(np.nan, 0)

ECR_data = all_ECR_data.loc[(all_ECR_data != 0).all(axis=1), :]
ECR_data

In [ ]:
# temp_max_l = list(temp_max[1])
# time_temp_l = [str('%.4f' % float(time)) +'s'+ ' MT: ' + str('%.2f' % temp) +'K' for time, temp in zip(columns, temp_max_l)]

In [ ]:
temp_path = os.path.join(mkpath, T_E_FILE+'.nlh') 
temp_max = pd.read_csv(temp_path,header=None,skiprows=6,skipfooter=2,sep = 2*'  ',engine='python', names=['TIME','TEMP'])
print(temp_max.head())
puls_name = 'I_NIMAK_0063'                     # TODO
puls_path =  'F:/SHK/Puls/'                              # TODO
puls = pd.read_csv(puls_path + puls_name + '.csv', header = None,sep=',',names=['TIME','PULS'])   
print(puls.head())
puls.TIME= puls.TIME + 2.029
temp_max.TIME = temp_max.TIME + 2.029
x1 = temp_max.TIME
x2 = puls.TIME
y1 = temp_max.TEMP
y2 = puls.PULS/1000 # A -> KA

fig, ax = plt.subplots(figsize=(14, 8))

ln1 = ax.plot(x2,y2,'#440154FF', label = 'Schweißstrom',lw = 3)
ax.set_ylabel('$I$ / KA', fontsize=17)
ax.set_xlabel('$t$ / s', fontsize=17)
ax.set_xlim(min(puls.TIME),max(puls.TIME))
ax.set_ylim(0)
ax2 = ax.twinx()
ln2 = ax2.plot(x1,y1,'#238A8DFF',label = 'Maximaltemperatur',lw=3)
ax2.set_ylabel('$T$$_{max}$ / K', fontsize=17)
ax2.set_ylim(298)
ax.grid()
ax.tick_params(labelsize=17)
ax2.tick_params(labelsize=17)
lns = ln1 + ln2
labs = [l.get_label() for l in lns]
ax.legend(lns,labs,loc = 0,fontsize=17)
title = 'Zeitlicher Verlauf der Maximaltemperatur (Ringbuckel 30°)'
plt.title(title,fontsize=20)
plt.show()

In [ ]:
%matplotlib notebook
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
x = ECR_data.index.to_list()
time_v = [float(time) for time in list(ECR_data)]
time_v_str = [str(time) for time in time_v]
n = len(list(ECR_data))

In [ ]:
X, Y, Z = [], [], []
for a in range(len(x)):
    idx = a
    for b in range(n):
        idy = b
        X.append(x[idx] * 1000)
        Z.append(ECR_data.iloc[idx, idy] )
        Y.append(time_v[idy])
X = np.array(X).reshape(len(x), n)
Y = np.array(Y).reshape(len(x), n)
Z = np.array( np.log10(Z)).reshape(len(x), n)

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(X,
                       Y,
                       Z,
                       cmap=plt.cm.viridis_r, 
                       alpha=0.8,
                       antialiased=False)
title = 'ECR mit Position bei unterischielicher Zeitpunkt  (unvollständig) '
ax.set_xlabel('X Koordi. in $mm$')
ax.set_ylabel('Zeit in $s$')
ax.set_zlabel('log (ECR) in $\u03A9.m^2$')
plt.title(title)
fig.colorbar(surf, shrink=0.5)
plt.show()

In [ ]:
import cv2
import cmapy
from PIL import Image

In [ ]:
filecode = 'therm_cdw_ring_al_py'           # TODO
cwd  =  'F:/SHK/Solution/'                    # TODO
mkpath = cwd + filecode
files = [file for file in [file for file in os.listdir('{}/'.format(mkpath)) if file.endswith('.png')] if file.split('_')[0] == 'Thermo']
fps = 10
size = (1077,810)
file_path = '{}/Thermo_Electric.mp4'.format(mkpath)
fourcc = cv2.VideoWriter_fourcc('H', '2', '6', '4')
video = cv2.VideoWriter( file_path, fourcc, fps,size )
for item in files:
    item = mkpath + '/' + item
    img = cv2.imread(item)
    img_colorized = cmapy.colorize(img, 'hot')
    video.write(img_colorized)
video.release()